In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

In [1]:
file = open('UBS_100_korrigiert_FINAL.csv', 'r')
liste = file.read()
Comp_List = liste.replace('\n', '').split(',')
Comp_List

#Hier gilt es anzumerken, dass einige Firmen fehlen, weil es keine Besitzstandsmeldungen gibt und auch Doppel-Einträge.
#Insgesamt resultieren darum nur 88 Firmen.

['ABB Ltd.',
 'Adecco Group AG',
 'Alcon Inc.',
 'Allreal Holding AG',
 'ALSO Holding AG',
 'ams AG',
 'ARYZTA AG',
 'Bâloise Holding AG',
 'Barry Callebaut AG',
 'Banque Cantonale Vaudoise',
 'BELIMO Holding AG',
 'BKW AG',
 'Bossard Holding AG',
 'Bucher Industries AG',
 'Burckhardt Compression Holding AG',
 'Cembra Money Bank AG',
 'Clariant AG',
 'Comet Holding AG',
 'Conzzeta AG',
 'Credit Suisse Group AG',
 'Dätwyler Holding AG',
 'DKSH Holding AG',
 'dormakaba Holding AG',
 'DUFRY AG',
 'Emmi AG',
 'Ems-Chemie Holding AG',
 'Georg Fischer AG',
 'Flughafen Zürich AG',
 'Forbo Holding AG',
 'Galenica AG',
 'GAM Holding AG',
 'Geberit AG',
 'Givaudan SA',
 'Helvetia Holding AG',
 'Huber+Suhner AG',
 'Implenia AG',
 'INFICON Holding AG',
 'Interroll Holding AG',
 'Intershop Holding AG',
 'Julius Bär Gruppe AG',
 'Jungfraubahn Holding AG',
 'Kardex AG',
 'Komax Holding AG',
 'Kühne + Nagel International AG',
 'LafargeHolcim Ltd',
 'Landis+Gyr Group AG',
 'LEM Holding SA',
 'Liechtens

In [74]:
#Dieses Skript greift die Daten aus der Datenbank ab und erstellt pro Firma ein CSV.
#Herausforderung waren insbesondere:
    #Ungeputzte Datenbank, es hatte im HTML immer noch scheinbar unzusammenhängende Einträge drin
    #schlecht geführte Datenbank: Die Firmen erstellen die Einträge selber, kaum Namens-, Struktur-, geschweige denn Adresskonventionen.
    #clickbare Untermenus, welche weitere Informationen liefern: Problem hier, Chrome Browser wurde als automatisiert erkannt und gestoppt. Lösung, Firefox(hiess aber auch, Code vielerorts anpassen, da sie nicht gleich funktionieren.)

#erster Teil des Loops greift die Besitzverhältnisse pro Firma ab und erstellt ersten Teil der DB.

for elem in Comp_List:

    
    driver = webdriver.Firefox(executable_path='/Users/fabianvonallmen/.wdm/drivers/geckodriver/v0.26.0/macos/geckodriver')
    
    driver.delete_all_cookies()
    
    driver.get('https://www.six-exchange-regulation.com/de/home/publications/significant-shareholders.html')
    
    #mal aktuelle beteiligungen anclicken
    driver.find_element_by_class_name('checkbox').click()
    #seriously? nur weil das oben war, nope. leider nein. 
    #das menu mal anwählen
    select = Select(driver.find_element_by_name('NotificationSubmitterId'))
    
    #jede Firma anwählen mit der Comp_List

    select.select_by_visible_text(elem)

    #UNTEN, alte Chrome-Befehle, der bei Firefox nicht funktioniert. Zur Sicherheit drin lassen, falls sich wieder was ändert in den Versionen.
    #enter = driver.find_element_by_name('NotificationSubmitterId').send_keys(Keys.ENTER)
    
    #driver.execute_script("window.scrollTo(1000, 400)") 
    #target = driver.find_element_by_class_name('clearfix')
    #target.location_once_scrolled_into_view

    #driver.find_element_by_class_name('pull-right').click()
    
    ####AND HERE, FIREFOX SPECIAL, PUSH THE BUTTON!
    time.sleep(3)
    #click 'Resultate anzeigen'
    driver.find_element_by_xpath("/html/body/div/div[5]/div[3]/div[1]/div[3]/div[1]/form/div/div[3]/button/span").click()
    

    #EIGENTLICH KÖNNTE MAN DAS STREICHEN, DIREKT WEITER OHNE ABSPEICHERN...zur Sicherheit lassen wirs drin.
    page = driver.page_source.encode('utf-8')
    with open(elem+'.htm', "wb+") as file:
        file.write(page)
        file.close()
        
    time.sleep(3)
    
    #und gleich wieder öffnen und beautifulsoupen
    file = open(elem+'.htm', 'r')
    text = file.read()
    abbh = BeautifulSoup(text, 'html.parser')
    
    #gleich suchradius definieren, als Grundlage für weiterere Arbeit
    tabelle = abbh.find_all('div', {'class': 'table-scroll'})
    
    #jetzt eine column finden um diese dann später einzeln durchzulaufen!

    column = tabelle[0].find_all('tr', {'class': 'itf-mark-row-template even'})
    column2 = tabelle[0].find_all('tr', {'class': 'itf-mark-row-template odd'})
    
    #das ganze mit odd UND even!!! Site so strukturiert, machts auch nicht einfacher. 
    #problem: offenbar sind da noch weitere Daten im HTML versteckt. Das ist gspässig und macht die sache nicht einfacher!
    #wtf?!
    #vielleicht müsste man die, die nicht dem suchbegriff "ABB" entsprechen mal schön löschen! das machen wir mit dem df.df issuer == ABB Ltd

    ergebnis = []
        
    for elem2 in column:
    
                DS = elem2.find('div', {'data-itf-if-filled': "ShareholderNames"}).get_text('/')
        #und kill 'mehr anzeigen'
                DS = DS.replace('/ Mehr anzeigen', '')
                Issuer = elem2.find('td', {'data-itf-inject': "NotificationSubmitter"}).text
                BO = elem2.find('div', {'data-itf-inject': "BeneficialNames"}).get_text('/')
                BO = BO.replace('/ Mehr anzeigen', '')
                Anteil = elem2.find('span', {'data-itf-inject': "PurchaseTotalVotingRate"}).text
    
                minidict = {'Issuer': Issuer, 
                            'BO': BO,
                            'DS': DS, 
                            'Anteil': Anteil}


                ergebnis.append(minidict)
                
    for elem3 in column2:
        
    
                DS = elem3.find('div', {'data-itf-if-filled': "ShareholderNames"}).get_text('/')
                DS = DS.replace('/ Mehr anzeigen', '')
                Issuer = elem3.find('td', {'data-itf-inject': "NotificationSubmitter"}).text
                BO = elem3.find('div', {'data-itf-inject': "BeneficialNames"}).get_text('/')
                BO = BO.replace('/ Mehr anzeigen', '')
                Anteil = elem3.find('span', {'data-itf-inject': "PurchaseTotalVotingRate"}).text
        
        
            
                minidict = {'Issuer': Issuer, 
                            'BO': BO,
                            'DS': DS, 
                            'Anteil': Anteil}
        
                ergebnis.append(minidict)
        
           
            
            
                df = pd.DataFrame(ergebnis)
                
                #eben, rest des gschmöis raus!
                df = df[df.Issuer == elem]
                
#PROBLEM:
                #elem and Issuer are not in synch!
                
            #something is not in sync here. It only gets ABB. Oh, oh, is the rest of the Website different? NOPE
            #must be a loop logic problem
            #nimmt nur das erste element, hmmm, bleibt stecken.
            #gelöst mit loop für column?
#SOLVED!!
                df1= pd.DataFrame(pd.DataFrame(df.BO.str.split('/').tolist(), index =df.Anteil).stack())
    
                df2= pd.DataFrame(pd.DataFrame(df.DS.str.split('/').tolist(), index =df.Anteil).stack())
    
                newdf = pd.merge(df1, df2, left_index=True, right_index=True, how='outer')
                newdf = newdf.dropna()
                newdf = newdf.replace('\n', np.nan)
                #columns umbenennen
                newdf = newdf.rename(columns={"0_x": "BO", "0_y": "DS"})
                #Name der Firma in neuer Column
                newdf.insert(0, 'Company', elem)

    
                #abspeichern für die konvertierung, sehr unelegant...
                newdf.to_csv(r'zwischen_'+elem +'.csv')
    
                #eben, öffnen aber zum glück nun ohne multiindex 
                dfliste_ohne = pd.read_csv('zwischen_'+elem +'.csv')
        
                #die folgende geniale funktion teilt die anteile für jede person einzeln auf, so dass man genau sehen kann, wer wie viel anteil hat. 
                #Achtung: Das kann aber nur eine statistische Annäherung sein, da nicht klar ist, ob die Anteile gleichwertig auf die verschiedenen Personen verteilt sind. 
                #Kann man aber begründen und machen, insbesondere um die Ländereinteilung mit Infos zu füttern. 
                
                news = (dfliste_ohne['Anteil'].str.replace('%', '').astype(float) / 
                      dfliste_ohne.groupby('Anteil')['BO'].
                      transform('count')).astype(str) + '%'
                #nach dem teilen wieder zusammenführen, divide et impera!
                dfliste_ohne['Anteil'] = news
                #ACHTUNG: HIER ZWISCHENSPEICHERN?
                dfliste_ohne.to_csv(elem+'_OHNE_Länder.csv')
                
        
                #scroll down
                target = driver.find_element_by_class_name('table-scroll')
                target.location_once_scrolled_into_view
                lst = driver.find_elements_by_class_name('fa.fa-info-circle.fa-2x')
    
    
                time.sleep(2)
                #####
        
#JETZT kommt der Unterlink mit den Ländern teil! 
#hier die verschiedenen einträge abclicken und ins untermenu rein. 
    Unterliste = []

    for clickloop in lst:
        try:                
            clickloop.click()
            time.sleep(2)
            #hier alles machen (INPUT 1)
            
            page = driver.page_source.encode('utf-8')
        
            #die ganze csv speicherei lasse ich hier mal bleiben, das mach ich besser mit append. 
        
            bs = BeautifulSoup(page, 'html.parser')    
                        #########
                        #muss ich das erst abspeichern, oder kann ich es einfach appenden? source encode und weiter im text.
                        #TRY!!!
        
                        #alle infos in dieser linie, kommen als schlauch raus, werden mit / in die jeweiligen Einträge geteilt. 
            adr = bs.find('td', {'data-itf-inject': 'BeneficialAddrs'}).get_text('/')
            adr = adr.split('/')
        
        
                        #alle infos in dieser linie, kommen als schlauch raus, werden mit / in die jeweiligen Einträge geteilt. 
            s = pd.Series(adr)
                        #von hier in Container speichern:
            Unterliste.append(s)
        
                        #ZURÜCK UND WIEDER VON VORNE 
            driver.back()
            time.sleep(6)
        except Exception:
             pass
            
                #master =  master.merge(lst,  how= 'outer') #master muss schon definiert sein
                
                ###
                    #all in one!
    U = pd.DataFrame(Unterliste).stack(dropna=True).str.split(pat = ",", expand=True)
                #make non to NaN
    U.fillna(value=pd.np.nan, inplace=True)

    this = pd.DataFrame(U.ffill(axis=1).iloc[:, -1])

                #holt den ersten Eintrag, dieser jeweils Name oder Firma. 
    this2 = U.iloc[:, [0]]

                #zusammenfügen, damit mit 0 Column verbunden werden kann. Eventuell Name ändern. 
    df_country = pd.concat([this2, this], axis=1, join='outer')

                #rename it, damit das verbunden werden kann mit master df
    df_country= df_country.rename(columns={0: "BO", df_country.columns[-1]: "Country"})
                
                #mal zwischenspeichern
    df_country.to_csv(elem+'__Länder.csv')
    
    #AND NOW:
    #ALLES ZUSAMMENFÜHREN UND ABSPEICHERN!!
    master =  dfliste_ohne.merge(df_country,  how= 'outer')
    master.to_csv(elem+'_MASTER.csv')
        
    driver.close()
            
           
             
                            



IndexError: list index out of range

In [43]:
page = driver.page_source.encode('utf-8')
        
            #die ganze csv speicherei lasse ich hier mal bleiben, das mach ich besser mit append. 
        
bs = BeautifulSoup(page, 'html.parser')    
                        #########
                        #muss ich das erst abspeichern, oder kann ich es einfach appenden? source encode und weiter im text.
                        #TRY!!!
        
                        #alle infos in dieser linie, kommen als schlauch raus, werden mit / in die jeweiligen Einträge geteilt. 
adr = bs.find('td', {'data-itf-inject': 'BeneficialAddrs'}).get_text('/')
adr = adr.split('/')
        
        
                        #alle infos in dieser linie, kommen als schlauch raus, werden mit / in die jeweiligen Einträge geteilt. 
s = pd.Series(adr)
                        #von hier in Container speichern:
Unterliste.append(s)
        
                        #ZURÜCK UND WIEDER VON VORNE 
driver.back()
time.sleep(6)
       
            
                        #master =  master.merge(lst,  how= 'outer') #master muss schon definiert sein
                
                ###
                    #all in one!
U = pd.DataFrame(Unterliste).stack(dropna=True).str.split(pat = ",", expand=True)
                #make non to NaN
U.fillna(value=pd.np.nan, inplace=True)

this = pd.DataFrame(U.ffill(axis=1).iloc[:, -1])

                #holt den ersten Eintrag, dieser jeweils Name oder Firma. 
this2 = U.iloc[:, [0]]

                #zusammenfügen, damit mit 0 Column verbunden werden kann. Eventuell Name ändern. 
df_country = pd.concat([this2, this], axis=1, join='outer')

            #rename it, damit das verbunden werden kann mit master df
df_country= df_country.rename(columns={0: "BO", df_country.columns[-1]: "Country"})
                
                #mal zwischenspeichern
df_country.to_csv(elem+'__Länder.csv')
    
    #AND NOW:
    #ALLES ZUSAMMENFÜHREN UND ABSPEICHERN!!
master =  dfliste_ohne.merge(df_country,  how= 'outer')
master.to_csv(elem+'_MASTER.csv')
        
driver.close()

In [20]:
file = open('UBS 100 Index.csv', 'r')
liste = file.read()
liste = liste.replace('\n', '').split(',')
liste

In [ ]:

Comp_List = ('ABB Ltd', 'Actelion Ltd')

for elem in Comp_List:
    #unbedingt fehlertoleranz einbauen, falls ein wort falsch in Comp_List oder Einträge leer!
    try: 

    
        driver = webdriver.Firefox(executable_path='/Users/fabianvonallmen/.wdm/drivers/geckodriver/v0.26.0/macos/geckodriver')
    
        driver.delete_all_cookies()
    
        driver.get('https://www.six-exchange-regulation.com/de/home/publications/significant-shareholders.html')
    
        #mal aktuelle beteiligungen anclicken
        driver.find_element_by_class_name('checkbox').click()
        #seriously? nur weil das oben war, nope. leider nein. 
        #das menu mal anwählen
        select = Select(driver.find_element_by_name('NotificationSubmitterId'))
    
        #jede Firma anwählen mit der Comp_List
        select.select_by_visible_text(elem)
        #enter = driver.find_element_by_name('NotificationSubmitterId').send_keys(Keys.ENTER)
    
        #driver.execute_script("window.scrollTo(1000, 0)") 
        #target = driver.find_element_by_class_name('clearfix')
        #target.location_once_scrolled_into_view

        #driver.find_element_by_class_name('pull-right').click()
    
        ####AND HERE, FIREFOX SPECIAL, PUSH THE BUTTON!
        time.sleep(3)
        #click 'Resultate anzeigen'
        driver.find_element_by_xpath("/html/body/div/div[5]/div[3]/div[1]/div[3]/div[1]/form/div/div[3]/button/span").click()
    
        time.sleep(1)
    
    
        #EIGENTLICH KÖNNTE MAN DAS STREICHEN, DIREKT WEITER OHNE ABSPEICHERN...
        page = driver.page_source.encode('utf-8')
        with open(elem+'.htm', "wb+") as file:
            file.write(page)
            file.close()
        
        time.sleep(3)
    
        ##driver.close()
    
        #und gleich wieder öffnen und beautifulsoupen
        file = open(elem+'.htm', 'r')
        text = file.read()
        abbh = BeautifulSoup(text, 'html.parser')
        
        #gleich suchradius definieren, als Grundlage für weiterere Arbeit
        tabelle = abbh.find_all('div', {'class': 'table-scroll'})
    
        #jetzt eine column finden um diese dann später einzeln durchzulaufen!
        #column
            
        #jetzt eine column finden um diese dann später einzeln durchzulaufen!
        #column
        #könnte man diesen Schritt nicht einfach, respektive den tabellenschritt einfach auslassen?
        column = tabelle[0].find_all('tr', {'class': 'itf-mark-row-template even'})
        column2 = tabelle[0].find_all('tr', {'class': 'itf-mark-row-template odd'})
    
        #das ganze mit odd UND even!!!
        #ausserdem werden die mehrfacheinträge schon mal mit 
        #problem: offenbar sind da noch weitere Daten im HTML versteckt. Das ist gspässig und macht die sache nicht einfacher!
        #wtf?!
        #vielleicht müsste man die, die nicht dem suchbegriff "ABB" entsprechen mal schön löschen! das machen wir mit dem df.df issuer == ABB Ltd

        ergebnis = []
    except Exception:
        pass
        
        for elem2 in column:
            try:
    
                    DS = elem2.find('div', {'data-itf-if-filled': "ShareholderNames"}).get_text('/')
        #und kill 'mehr anzeigen'
                    DS = DS.replace('/ Mehr anzeigen', '')
                    Issuer = elem2.find('td', {'data-itf-inject': "NotificationSubmitter"}).text
                    BO = elem2.find('div', {'data-itf-inject': "BeneficialNames"}).get_text('/')
                    BO = BO.replace('/ Mehr anzeigen', '')
                    Anteil = elem2.find('span', {'data-itf-inject': "PurchaseTotalVotingRate"}).text
    
                    minidict = {'Issuer': Issuer, 
                            'BO': BO,
                            'DS': DS, 
                            'Anteil': Anteil}
            except Exception:
                pass

                ergebnis.append(minidict)
                
        for elem3 in column2:
            try:
        
                    DS = elem3.find('div', {'data-itf-if-filled': "ShareholderNames"}).get_text('/')
                    DS = DS.replace('/ Mehr anzeigen', '')
                    Issuer = elem3.find('td', {'data-itf-inject': "NotificationSubmitter"}).text
                    BO = elem3.find('div', {'data-itf-inject': "BeneficialNames"}).get_text('/')
                    BO = BO.replace('/ Mehr anzeigen', '')
                    Anteil = elem3.find('span', {'data-itf-inject': "PurchaseTotalVotingRate"}).text
            
                    minidict = {'Issuer': Issuer, 
                            'BO': BO,
                            'DS': DS, 
                            'Anteil': Anteil}
            except Exception:
                pass
        
                ergebnis.append(minidict)
            
                df = pd.DataFrame(ergebnis)
                
                df = df[df.Issuer == elem]
                
#SOLVED!!
                #elem and Issuer are not in synch!
                
                 #something is not in sync here. It only gets ABB. Oh, oh, is the rest of the Website different? NOPE
            #must be a loop logic problem
            #nimmt nur das erste element, hmmm, bleibt stecken.
            #gelöst mit loop für column?
#SOLVED!!
                df1= pd.DataFrame(pd.DataFrame(df.BO.str.split('/').tolist(), index =df.Anteil).stack())
    
                df2= pd.DataFrame(pd.DataFrame(df.DS.str.split('/').tolist(), index =df.Anteil).stack())
    
                newdf = pd.merge(df1, df2, left_index=True, right_index=True, how='outer')
                newdf = newdf.dropna()
                newdf = newdf.replace('\n', np.nan)
                #columns umbenennen
                newdf = newdf.rename(columns={"0_x": "BO", "0_y": "DS"})
                #Name der Firma in neuer Column
                newdf.insert(0, 'Company', elem)

    
                #abspeichern für die konvertierung, sehr unelegant...
                newdf.to_csv(r'zwischen_'+elem +'.csv')
    
                #eben, öffnen aber zum glück nun ohne multiindex 
                dfliste_ohne = pd.read_csv('zwischen_'+elem +'.csv')
        
                #diese geniale funktion teilt die anteile für jede person einzeln auf, so dass 
                news = (dfliste_ohne['Anteil'].str.replace('%', '').astype(float) / 
                          dfliste_ohne.groupby('Anteil')['BO'].
                          transform('count')).astype(str) + '%'
                    #nach dem teilen wieder zusammenführen, divide et impera!
                dfliste_ohne['Anteil'] = news
                    #ACHTUNG: HIER ZWISCHENSPEICHERN?
                dfliste_ohne.to_csv(elem+'_OHNE_Länder.csv')
    
        
                #JETZT kommt der Unterlink mit den Ländern teil!
                #driver
                
                #just tryin if iloc works
               
                    
#hier die verschiedenen einträge abclicken und ins untermenu rein. 
                
        
        #scroll down
                target = driver.find_element_by_class_name('table-scroll')
                target.location_once_scrolled_into_view
    
    #ABER JETZT ERST NOCH DIE UNTERMENUS LÄNDER ERSTELLEN
                lst = driver.find_elements_by_class_name('fa.fa-info-circle.fa-2x')
    
    
                time.sleep(2)
                #####

    
        Unterliste = []

    for clickloop in lst:
        try:                
            clickloop.click()
            time.sleep(2)
            #hier alles machen (INPUT 1)
            
            page = driver.page_source.encode('utf-8')
        
            #die ganze csv speicherei lasse ich hier mal bleiben, das mach ich besser mit append. 
        
            bs = BeautifulSoup(page, 'html.parser')    
                        #########
                        #muss ich das erst abspeichern, oder kann ich es einfach appenden? source encode und weiter im text.
                        #TRY!!!
        
                        #alle infos in dieser linie, kommen als schlauch raus, werden mit / in die jeweiligen Einträge geteilt. 
            adr = bs.find('td', {'data-itf-inject': 'BeneficialAddrs'}).get_text('/')
            adr = adr.split('/')
        
        
                        #alle infos in dieser linie, kommen als schlauch raus, werden mit / in die jeweiligen Einträge geteilt. 
            s = pd.Series(adr)
                        #von hier in Container speichern:
            Unterliste.append(s)
        
                        #ZURÜCK UND WIEDER VON VORNE 
            driver.back()
            time.sleep(6)
        except Exception:
             pass
            
                        #master =  master.merge(lst,  how= 'outer') #master muss schon definiert sein
                
                ###
                    #all in one!
    U = pd.DataFrame(Unterliste).stack(dropna=True).str.split(pat = ",", expand=True)
                #make non to NaN
    U.fillna(value=pd.np.nan, inplace=True)

    this = pd.DataFrame(U.ffill(axis=1).iloc[:, -1])

                #holt den ersten Eintrag, dieser jeweils Name oder Firma. 
    this2 = U.iloc[:, [0]]

                #zusammenfügen, damit mit 0 Column verbunden werden kann. Eventuell Name ändern. 
    df_country = pd.concat([this2, this], axis=1, join='outer')

                #rename it, damit das verbunden werden kann mit master df
    df_country= df_country.rename(columns={0: "BO", df_country.columns[-1]: "Country"})
                
                #mal zwischenspeichern
    df_country.to_csv(elem+'__Länder.csv')
    
    #AND NOW:
    #ALLES ZUSAMMENFÜHREN UND ABSPEICHERN!!
    master =  dfliste_ohne.merge(df_country,  how= 'outer')
    master.to_csv(elem+'_MASTER.csv')
        
    driver.close()
            
           
             
                            
